# Jupyter notebook

This is a Jupyter notebook, an interactive website you can edit, and in which you can run code. There are a lot of tutorials for Jupyter online ([here is one](https://realpython.com/jupyter-notebook-introduction/)); to execute cells you need to type `shift + enter`.

If you click on the "hamburger" menu on the left hand side you will get a table of contents.

# Brightway community

**Save the date**: [Brightcon 2022 will be held Sept. 26-30 in Luxembourg](https://2022.brightcon.link/).

**Keep in touch**: Sign up for the [Brightcon mailing list](https://brightway.groups.io/g/brightcon/) and [Brightway updates](https://brightway.groups.io/g/updates/)

# Importing the libraries needed for this demo

We start with importing three Brightway libraries: [bw_processing](https://github.com/brightway-lca/bw_processing) [matrix_utils](https://github.com/brightway-lca/matrix_utils), and [bw2calc](https://github.com/brightway-lca/brightway2-calc).

In [ ]:
import bw_processing as bwp
import matrix_utils as mu
import bw2calc as bc
import numpy as np
import seaborn as sb
import pandas as pd

# Common example

In this notebook, we will use this example product system:

<img src='images/simple-graph.png' width='400'>

We need to assign integer ids to each node:

In [ ]:
natural_gas = 101
carbon_fibre = 102
bike = 103
co2 = 201

In our technosphere matrix, we will have three production exchanges (each of amount 1), and two consumption exchanges. Our biosphere matrix will only have one number, the emission of $CO_{2}$ from carbon fibre production.

Our matrices should look like this:

## Technosphere matrix

Dimensions are products (rows) by activities (columns).

$$
\left[\begin{array}{ccc} 
1 & 0 & 0\\
-237 & 1 & 0\\
0 & -2.5 & 1\\
\end{array}\right]
$$

## Biosphere matrix

Dimensions are flows (rows) by activities (columns).

$$
\left[\begin{array}{ccc} 
0 & 26.6 & 0\\ 
\end{array}\right]
$$ 

## Characterization matrix

Dimensions are flows (rows) by flows (columns).

$$
\left[\begin{array}{c} 
1\\ 
\end{array}\right]
$$

# A first LCA calculation

To use datapackages, we need to know four thing for each point: the row and column ids, the data value, and the sign. We only need to store the non-zero points. Here is an example for the technosphere matrix:

In [ ]:
t_data = np.array([1, 1, 1, 237, 2.5])
t_indices = np.array([
    (101, 101), # production of natural gas
    (102, 102), # production of carbon fibre
    (103, 103), # production of bike
    (101, 102), # input of natural gas
    (102, 103), # input of carbon fibre
    ], 
    dtype=bwp.INDICES_DTYPE
)
t_flip = np.array([False, False, False, True, True]) # Numerical sign of the inputs needs to be flipped negative

And similarly for the other matrices (no need to flip signs, so we skip that part):

In [ ]:
b_data = np.array([26.6])
b_indices = np.array([
    (201, 102), # emission of CO2
    ], 
    dtype=bwp.INDICES_DTYPE
)

In [ ]:
c_data = np.array([1])
c_indices = np.array([
    (201, 201), # CF of CO2
    ], 
    dtype=bwp.INDICES_DTYPE
)

We can now create our datapackage, and add the data for all three matrices to it:

In [ ]:
dp_static = bwp.create_datapackage()

In [ ]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=t_indices,
    data_array=t_data,
    flip_array=t_flip,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_indices,
    data_array=b_data,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_indices,
    data_array=c_data,
)

This is already enough to calculate an LCA score:

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static],
)
lca.lci()
lca.lcia()
lca.score

# A stochastic LCA

This is pretty boring so far. The only marginally interesting thing is that we could put our datapackage on another computer, and do calculations on that computer - i.e. we are ready for cloud calculations. Indeed, the library `matrix_utils` allows us to specify datapackages on FTP sites and cloud storage such as Amazon S3.

But we can make our example more interesting by adding uncertainty. To do this, we will use the standards in the [stats_arrays](https://stats-arrays.readthedocs.io/en/latest/) library.

We will only add uncertainty to the inputs, and the CO2 emission. The other edges will have uncertainty type zero - no uncertainty.

In [ ]:
t_uncertainty = np.array([
        (0, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),
        (0, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),    
        (0, 1, np.NaN, np.NaN, np.NaN, np.NaN, False),    
        (5, 237, np.NaN, np.NaN, 200, 300, False), # triangular uncertainty from 200 to 300  
        (5, 2.5, np.NaN, np.NaN, 2, 3, False), # triangular uncertainty from 2 to 3
    ], 
    dtype=bwp.UNCERTAINTY_DTYPE
)
b_uncertainty = np.array([
        (3, 26.6, 1.5, np.NaN, np.NaN, np.NaN, False), # normal uncertainty with std. dev. of 1.5
    ], 
    dtype=bwp.UNCERTAINTY_DTYPE
)

We then will create another data package, and use it to do Monte Carlo:

In [ ]:
dp_stochastic = bwp.create_datapackage()

In [ ]:
dp_stochastic.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=t_indices,
    data_array=t_data,
    flip_array=t_flip,
    distributions_array=t_uncertainty,
)
dp_stochastic.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_indices,
    data_array=b_data,
    distributions_array=b_uncertainty,
)
dp_stochastic.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_indices,
    data_array=c_data,
)

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_stochastic],
    use_distributions=True,
)
lca.lci()
lca.lcia()
    
mc_results = [lca.score for _ in zip(range(100), lca)]

In [ ]:
sb.displot(mc_results)

# Correlating exchanges

If we sample each exchange independently, we lose some of the logic of our product system. In our example system, the $CO_{2}$ should be tied directly to the natural gas consumption, but it isn't:

In [ ]:
params = []

for _ in range(100):
    next(lca)
    params.append({
        'co2': lca.biosphere_matrix[lca.dicts.biosphere[co2], lca.dicts.activity[carbon_fibre]],
        'ng': -1 * lca.technosphere_matrix[lca.dicts.product[natural_gas], lca.dicts.activity[carbon_fibre]],
    })

In [ ]:
sb.scatterplot(data=pd.DataFrame(params), x='ng', y='co2')

We fix this by drawing correlated samples ahead of time - this is the idea behind the `presamples` package, whose functionality is now integrated directly into Brightway. We will make up a model to illustrate how this would work; we can assume that (again, totally made up) 80% of the natural gas goes to energy, the ratio stays the same, but with a little bit of noise.

In [ ]:
ng_samples = np.random.triangular(200, 237, 300, size=100)
co2_samples = 26.6 / 237 * ng_samples * np.random.normal(loc=1, scale=0.025, size=100)

We can now add another datapackage which will overwrite our previous values (but only where we tell it to). Note that we are now creating `arrays`, not `vectors`.

We need to tell the datapackage that these two resources are correlated - otherwise there would be two RNGs used to samples them independently again. We can do this by either setting `sequential=True` (in which case they would start with column 0, then column 1, etc.), or by using the same RNG seed for both resources.

In [ ]:
dp_correlated = bwp.create_datapackage(seed=42)

In [ ]:
dp_correlated.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(101, 102)], dtype=bwp.INDICES_DTYPE),
    data_array=ng_samples.reshape((1, -1)),
    flip_array=np.array([True]) ,
)
dp_correlated.add_persistent_array(
    matrix='biosphere_matrix',
    indices_array=np.array([(201, 102)], dtype=bwp.INDICES_DTYPE),
    data_array=co2_samples.reshape((1, -1)),
)

And now the use of our correlated samples reflects reality a lot better. Note that we justs add the new datapackage after the original one, and set `use_arrays=True`:

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_stochastic, dp_correlated],
    use_distributions=True,
    use_arrays=True,
)
lca.lci()
lca.lcia()

corr_params = []

for _ in range(100):
    next(lca)
    corr_params.append({
        'co2': lca.biosphere_matrix[lca.dicts.biosphere[co2], lca.dicts.activity[carbon_fibre]],
        'ng': -1 * lca.technosphere_matrix[lca.dicts.product[natural_gas], lca.dicts.activity[carbon_fibre]],
    })

In [ ]:
sb.scatterplot(data=pd.DataFrame(corr_params), x='ng', y='co2')

# Using arrays for scenarios

In addition to correlated samples, we can use arrays for scenarios. Let's imagine two different possibilities: a lightweight bike with 1.5 kilograms of carbon fibre, and a technology platform that allows for efficient bike sharing making each bike functionally equivalent to two bikes (I know, I find this silly as well). If we treat these as separate possibilities, we have four scenarios in total.

For this to work, we will need to create **two** new arrays, one for each choice, and then tell the software to do combinatorial sampling:

In [ ]:
dp_scenarios = bwp.create_datapackage(combinatorial=True)

In [ ]:
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(102, 103)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([(2.5, 1.5)]),
    flip_array=np.array([True]),
    name='cf scenario'
)
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(103, 103)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([(1, 2)]),
    name='double bike'
)

In [ ]:
scenario_mapping = {
    (0, 0): "Original",
    (0, 1): "Each bike counts double",
    (1, 0): "Lightweight",
    (1, 1): "Lightweight & each bike counts double",
}

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static, dp_scenarios],
    use_arrays=True,
)
lca.lci()
lca.lcia()

In [ ]:
resource_group = next(grp for grp in lca.technosphere_mm.groups if grp.label == 'double bike').indexer.indexer

In [ ]:
print(lca.score, scenario_mapping[resource_group.index])

for scenario_result in lca:
    print(lca.score, scenario_mapping[resource_group.index])

# Interfaces

We can add some electricity consumption to the bike manufacturing process, and demonstrate how an `Interface` should work. For our example interface, we can use the Danish company Energinet's [API for the 5 minute interval CO2 intensity of Danish electricity](https://www.energidataservice.dk/tso-electricity/co2emis):

In [ ]:
import requests

class DenmarkCO2Interface:
    size = (1,)

    def __next__(self):
        URL = 'https://api.energidataservice.dk/datastore_search?resource_id=co2emis&limit=1&sort=Minutes5UTC%20desc'
        print("Querying energidataservice.dk API")
        result = requests.get(URL).json()['result']['records'][0]['CO2Emission']
        return np.array([result]) / 1000 # g to kg

That was just the CO2 intensity, we also need to add a node for electricity consumption. We will give it the ID `104`, and use 100 kilowatt hours (way too high, I know).

In [ ]:
dk_elec_dp = bwp.create_datapackage()

In [ ]:
dk_elec_dp.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=np.array([
            (104, 104), # production of electricity
            (104, 103), # use of electricity in bike production
        ], 
        dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.array([1, 100]),
    flip_array=np.array([False, True]),
)

We can now add our interface. We are not adding `persistent` data, but `dynamic` data:

In [ ]:
dk_elec_dp.add_dynamic_vector(
    matrix='biosphere_matrix',
    interface=DenmarkCO2Interface(),
    indices_array=np.array([(201, 104)], dtype=bwp.INDICES_DTYPE),
)

As before, we can just add the data packages together:

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static, dk_elec_dp],
)
lca.lci()
lca.lcia()
lca.score

This is the value we got for CO2 intensity (in kg CO2-eq/kwh):

In [ ]:
lca.biosphere_matrix[0, lca.dicts.activity[104]]

Other countries also have similar APIs, for example France:

<div class="alert alert-block alert-warning">
    The Interface below includes a parameter <code>verify=False</code> in the call to the <code>requests</code> library to pull the data.
    
    
The TLS configuration from www.rte-france.com is incomplete, but the certificates are valid. You can verify the validity by pointing your browser to the <a href="https://www.rte-france.com/themes/swi/xml/power-co2-emission-fr.xml"> API URL</a> <strong>. 
    We can safely use requests with: </strong> <code>"verify=False"</code>, but the <code>requests</code> library will still warn us about this.
    
More details on this at https://github.com/Depart-de-Sentier/try_brightway_dev/issues/3

</div>

In [ ]:
from time import time
import xml.etree.ElementTree as ET

class FranceCO2Interface:
    size = (1,)
    
    def __next__(self):
        # Note: has timestamp but seems to always give latest data
        URL_TEMPLATE = "https://www.rte-france.com/themes/swi/xml/power-co2-emission-fr.xml?_={}"
        print("Querying RTE Eco2mix interface")
        URL = URL_TEMPLATE.format(int(time()))
        # the TLS configuration from www.rte-france.com is incomplete, but the certificates are valid.
        # To verify the validity, point your browser to the URL
        # so, we can safely use requests with "verify=False"
        # see issue on the repo of this notebook -> https://github.com/Depart-de-Sentier/try_brightway_dev/issues/3
        # requests will still warn us about this.
        resp = requests.get(URL, verify=False)
        root = ET.fromstring(resp.text)
        return np.array([[float(child.text) for child in root.iter('valeur')][-1]]) / 1000 # g to kg

In [ ]:
fr_elec_dp = bwp.create_datapackage()

fr_elec_dp.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=np.array([
            (104, 104), # production of electricity
            (104, 103), # use of electricity in bike production
        ], 
        dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.array([1, 100]),
    flip_array=np.array([False, True]),
)

fr_elec_dp.add_dynamic_vector(
    matrix='biosphere_matrix',
    interface=FranceCO2Interface(),
    indices_array=np.array([(201, 104)], dtype=bwp.INDICES_DTYPE),
)

As before, we can just add the data packages together:

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static, fr_elec_dp],
)
lca.lci()
lca.lcia()
lca.score

This is the value we got for CO2 intensity (in kg CO2-eq/kwh):

In [ ]:
lca.biosphere_matrix[0, lca.dicts.activity[104]]